In [ ]:
import json
import platform
import time
import threading
import paho.mqtt.client as mqtt
import psutil

def conectar_mqtt(host, port, topic, mqtt_client):
    mqtt_client.connect(host, port, 60)
    mqtt_client.loop_start()

def obtener_info_sistema():
    try:
        system_info = {
            "Sistema": platform.system(),
            "Release": platform.release(),
            "Procesador": platform.processor(),
            "Nombre Sistema Operativo": platform.system_alias(platform.system(), platform.release(), 0),
            "Arquitectura": platform.architecture()[0]  # Obtener la arquitectura (32 bits o 64 bits)
        }
        return system_info
    except Exception as e:
        print(f"Error al obtener información del sistema: {e}")
        return None

def enviar_datos_maquina1(mqtt_client, mqtt_topic):
    while True:
        cpu_percent = psutil.cpu_percent(interval=1)
        ram_percent = psutil.virtual_memory().percent
        rom_percent = psutil.disk_usage('/').percent
        info_sistema = obtener_info_sistema()
        machine_id = "maquina1"

        datos_maquina1 = {
            "id_maquina": machine_id,
            "datos": {
                "CPU": cpu_percent,
                "RAM": ram_percent,
                "ROM": rom_percent,
                "Info_Sistema": info_sistema,
                "fecha": time.strftime('%Y-%m-%d %H:%M:%S')
            }
        }

        # Enviar los datos de la Máquina 1 a la Máquina 2
        mqtt_client.publish(mqtt_topic, json.dumps(datos_maquina1))
        time.sleep(1)

# Configuración MQTT
mqtt_host = "broker.hivemq.com"
mqtt_port = 1883
mqtt_topic = "grupo_7"
mqtt_client_maquina1 = mqtt.Client()
conectar_mqtt(mqtt_host, mqtt_port, mqtt_topic, mqtt_client_maquina1)

# Iniciar hilo para enviar datos desde la Máquina 1 a la Máquina 2
enviar_datos_maquina1_thread = threading.Thread(target=enviar_datos_maquina1, args=(mqtt_client_maquina1, mqtt_topic))
enviar_datos_maquina1_thread.start()

# Mantener el script principal en ejecución
enviar_datos_maquina1_thread.join()
